In [2]:
import datetime
import logging
import os
import numpy as np
import torch
from importlib import reload

from models import dapm
from scripts.data_loader import *
from scripts.train_dapm import train
from utils.metrics import normalize_mat
from params import Param
from utils.logging_utils import *

import warnings
warnings.filterwarnings('ignore')


In [3]:

def dapm_main(param, **kwargs):
    
    """ define model name """ 
    model_name = f'{param.area}_{param.resolution}m_{param.year}___#{param.months[0]}#'
    ae_model_name = param.generate_ae_model_name()
    print(model_name)
    print(ae_model_name)

    kwargs['model_name'] = model_name
    kwargs['model_file'] = os.path.join(kwargs['model_dir'], model_name + '.pkl')
    kwargs['log_file'] = os.path.join(kwargs['log_dir'], model_name + '.log')
    kwargs['run_file'] = os.path.join(kwargs['run_dir'], model_name + '_run_{}'.format(datetime.datetime.now().strftime('%d%H%m')))
    kwargs['ae_model_file'] = os.path.join('./data/ae_models/models/', ae_model_name + '.pkl')

    """ load data """
    data_dir = f'/home/yijun/notebooks/training_data/'
    data_obj = load_data(data_dir, param)
    train_loc, val_loc, test_loc = load_locations(kwargs['train_val_test'], param)
    
    data_obj.train_loc = train_loc
    data_obj.train_y = data_obj.gen_train_val_test_label(data_obj.label_mat, data_obj.train_loc)
    data_obj.val_loc = val_loc
    data_obj.val_y = data_obj.gen_train_val_test_label(data_obj.label_mat, data_obj.val_loc)
    data_obj.test_loc = test_loc
    data_obj.test_y = data_obj.gen_train_val_test_label(data_obj.label_mat, data_obj.test_loc)
    
    """ logging starts """
    start_logging(kwargs['log_file'], model_name)
    data_logging(data_obj)

    """ load ae model """
    ae = torch.load(kwargs['ae_model_file'])
    
    """ define DeepAP model
    in_dim, ae_en_h_dims, ae_de_h_dims
    conv_lstm_in_size, conv_lstm_in_dim, conv_lstm_h_dim, conv_lstm_kernel_sizes, conv_lstm_n_layers
    fc_in_dim, fc_h_dims, fc_out_dim  """
    m = dapm.DeepAPM(in_dim=data_obj.n_features,
                     ae_en_h_dims=param.ae_en_h_dims,
                     ae_de_h_dims=param.ae_de_h_dims,
                               
                     conv_lstm_in_size=(data_obj.n_rows, data_obj.n_cols),
                     conv_lstm_in_dim=param.ae_en_h_dims[-1],  
                     conv_lstm_h_dim=[param.dapm_h_dim],  # dap_h_dim
                     conv_lstm_kernel_sizes=param.kernel_sizes,  # kernel_sizes
                     conv_lstm_n_layers=1,
                               
                     fc_in_dim=param.dapm_h_dim * len(param.kernel_sizes),
                     fc_h_dims=param.fc_h_dims,  # fc_h_dims
                     fc_out_dim=1,
                                    
                     ae_pretrain_weight=ae.state_dict(),
                     mask_thre=param.mask_thre,
                     fc_p_dropout=0.1,
                     device=kwargs['device'])
   
    m = m.to(kwargs['device'])
    train(m, data_obj, param, **kwargs)
    
    """ logging ends """
    end_logging(model_name)
    

In [4]:
"""
    define directory
"""

base_dir = f'data/dapm_models/'
train_val_test_file = f'/home/yijun/notebooks/training_data/train_val_test_los_angeles_500m_fine_tune_1234.json'
device = torch.device("cuda:3" if torch.cuda.is_available() else 'cpu')  # the gpu device

""" load train, val, test locations """
f = open(train_val_test_file, 'r')
train_val_test = json.loads(f.read())

kwargs = {
    'model_dir': os.path.join(base_dir, 'models/'),
    'log_dir': os.path.join(base_dir, 'logs/'),
    'run_dir': os.path.join(base_dir, 'runs/'),
    'train_val_test': train_val_test,
    'device': device
}


In [6]:
for m in range(1, 13):
    param = Param([m], 2018, alpha=1, beta=0.1, gamma=5, eta=0, epochs=5, sp_neighbor=1, mask_thre=0.0001, lr=0.001, model_type=['sp', 'ae', 'sc'])
    dapm_main(param, **kwargs)           


los_angeles_500m_2018___#01#
ae___los_angeles_500m_2018___#01#___16
los_angeles_500m_2018___#02#
ae___los_angeles_500m_2018___#02#___16
los_angeles_500m_2018___#03#
ae___los_angeles_500m_2018___#03#___16
los_angeles_500m_2018___#04#
ae___los_angeles_500m_2018___#04#___16
los_angeles_500m_2018___#05#
ae___los_angeles_500m_2018___#05#___16
los_angeles_500m_2018___#06#
ae___los_angeles_500m_2018___#06#___16
los_angeles_500m_2018___#07#
ae___los_angeles_500m_2018___#07#___16
los_angeles_500m_2018___#08#
ae___los_angeles_500m_2018___#08#___16
los_angeles_500m_2018___#09#
ae___los_angeles_500m_2018___#09#___16
los_angeles_500m_2018___#10#
ae___los_angeles_500m_2018___#10#___16
los_angeles_500m_2018___#11#
ae___los_angeles_500m_2018___#11#___16
los_angeles_500m_2018___#12#
ae___los_angeles_500m_2018___#12#___16
